In [1]:
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
from utils.training import numpy_metrics

In [3]:
import glob

In [4]:
def softmax(x, axis=1):
    return np.exp(x) / np.sum(np.exp(x),axis=axis, keepdims=True)

In [5]:
def ece(outputs, labels, name="", bins=np.arange(0.1, 1.05, 0.05), print_name=False, background_label = 11):

    if outputs is None:
        ece = None
    else:
        confidences = np.max(outputs, 1)
        predictions = np.argmax(outputs,1)
        bin_lowers = bins[:-1]
        bin_uppers = bins[1:]
        
        accuracies = predictions == labels 

        #ece = Variable(torch.zeros(1)).type_as(confidences)
        ece = 0.0
        for bin_lower, bin_upper in zip(bin_lowers, bin_uppers):
            # Calculated |confidence - accuracy| in each bin
            in_bin = (confidences > bin_lower) * (confidences < bin_upper)
            prop_in_bin = in_bin.mean()
            if prop_in_bin > 0:
                accuracy_in_bin = accuracies[in_bin].mean()
                avg_confidence_in_bin = confidences[in_bin].mean()
                ece += np.abs(avg_confidence_in_bin-accuracy_in_bin) * prop_in_bin
    if print_name:
        print(name, end=": ")
    if ece is not None:
        print("%.3f"%ece)
    else:
        print("-")
    return ece

In [12]:
def compute_mIOU_acc(loc, output_probs = True, background_class = 11):
    npz_arr = np.load(loc)
    
    if output_probs:
        pred_probs = npz_arr['predictions']
    else:
        pred_probs = softmax(npz_arr['preds'],1)
    targets = npz_arr['targets']
    
    I, U, acc = numpy_metrics(pred_probs, targets)
    mIOU = np.mean(I / U)
    
    filtered_targets = targets.reshape(-1)
    background_labels = filtered_targets == background_class
    filtered_targets = filtered_targets[~background_labels]
    
    filtered_pred_probs = pred_probs.reshape(-1, pred_probs.shape[1])
    print(filtered_pred_probs.shape, filtered_pred_probs.sum(1).mean())
    filtered_pred_probs = filtered_pred_probs[~background_labels, :]
    
    ece_val = ece(filtered_pred_probs, filtered_targets, print_name=False)
    print(acc, mIOU, ece_val, loc)
    return acc, mIOU, ece_val
    

In [13]:
for f in glob.glob('test_preds/*/preds.npz'):
    print('Now running ', f.split('/')[1])
    compute_mIOU_acc(f)

Now running  swag_v2
(40262400, 11) 1.0000000163092146
0.091
0.90122923316984 0.5963490447443337 0.09119710592318797 test_preds/swag_v2/preds.npz
Now running  drop_v3
(40262400, 11) 1.0000000059618455
0.088
0.8582369743529388 0.5730634901020127 0.08765812476349522 test_preds/drop_v3/preds.npz
Now running  swag_v3
(40262400, 11) 1.0000000167198195
0.091
0.8973113909374301 0.5836219739910562 0.09101870187080591 test_preds/swag_v3/preds.npz
Now running  swag_v4
(40262400, 11) 1.000000017221361
0.091
0.8940361786560852 0.579722273512933 0.09132273007598482 test_preds/swag_v4/preds.npz
Now running  swag_scale_05
(40262400, 11) 1.000000016357578
0.091
0.9012262630531142 0.5970193596489862 0.09130706941536933 test_preds/swag_scale_05/preds.npz
Now running  swag_scale_05_v2
(40262400, 11) 1.0000000167893077
0.091
0.8968514360785554 0.5835723035447599 0.091116751596064 test_preds/swag_scale_05_v2/preds.npz
Now running  swag_scale_05_v3
(40262400, 11) 1.0000000173368202
0.091
0.894291608694508 0

In [14]:
compute_mIOU_acc('test_preds/dropout/dropout.npz')

(40262400, 11) 1.00000000589817
0.089
0.9022376007118156 0.6087888665322777 0.08900116104908891 test_preds/dropout/dropout.npz


(0.9022376007118156, 0.6087888665322777, 0.08900116104908891)

In [15]:
compute_mIOU_acc('test_preds/swag_cov/swag_50.npz')

(40262400, 11) 1.0000000158994804
0.092
0.9073940591002091 0.6161459621346941 0.09157205975762094 test_preds/swag_cov/swag_50.npz


(0.9073940591002091, 0.6161459621346941, 0.09157205975762094)

In [16]:
dropout_locs = ['test_preds/drop_v2/preds.npz', 'test_preds/drop_v3/preds.npz']
swagdiag_locs = ['test_preds/swag_diag/preds.npz', 'test_preds/swag_diag_v2/preds.npz', 
                 'test_preds/swag_diag_v3/preds.npz']
swag_locs = ['test_preds/swag_scale_05/preds.npz', 'test_preds/swag_scale_05_v2/preds.npz', 
             'test_preds/swag_scale_05_v3/preds.npz']
swa_locs = ['test_preds/sgd/swa_preds.npz', 'test_preds/sgd/swa_1000_v2.npz', 'test_preds/sgd/swa_1000_v3.npz']
sgd_locs = ['test_preds/sgd/sgd_preds.npz', 'test_preds/sgd/sgd_1000_v2.npz', 'test_preds/sgd/sgd_1000_v3.npz']


In [13]:
for d in dropout_locs:
    compute_mIOU_acc(d)

0.040
0.8405242312840742 0.5430310970106214 0.040217998904142724 test_preds/drop_v2/preds.npz
0.040
0.8582369743529388 0.5730634901020127 0.040161222809092276 test_preds/drop_v3/preds.npz


In [15]:
for sd in swagdiag_locs:
    compute_mIOU_acc(sd)

0.085
0.90122923316984 0.5963490447443337 0.08527136570525475 test_preds/swag_diag/preds.npz
0.089
0.897311416764532 0.5836220246543136 0.0885331741435351 test_preds/swag_diag_v2/preds.npz
0.089
0.89395838742497 0.5794386135626851 0.08909606774016379 test_preds/swag_diag_v3/preds.npz


In [16]:
for loc in swag_locs:
    compute_mIOU_acc(loc)

0.087
0.9012262630531142 0.5970193596489862 0.0873648097786202 test_preds/swag_scale_05/preds.npz
0.091
0.8968514360785554 0.5835723035447599 0.09120995396217654 test_preds/swag_scale_05_v2/preds.npz
0.091
0.894291608694508 0.5806040031650241 0.09107189907951052 test_preds/swag_scale_05_v3/preds.npz


In [17]:
for loc in sgd_locs:
    compute_mIOU_acc(loc, output_probs = False)

(40262400, 11) 1.000001
0.090
0.9055111600586347 0.6170782768632335 0.08959087652465114 test_preds/sgd/sgd_preds.npz
(40262400, 11) 0.9999986
0.090
0.9065664296177767 0.6324223625404213 0.08964456501753963 test_preds/sgd/sgd_1000_v2.npz


FileNotFoundError: [Errno 2] No such file or directory: 'test_preds/sgd/sgd_1000_v3.npz'

In [18]:
for loc in swa_locs:
    compute_mIOU_acc(loc, output_probs = False)

(40262400, 11) 0.9999976
0.092
0.9061910326907318 0.632293765243613 0.09200716841388291 test_preds/sgd/swa_preds.npz
(40262400, 11) 0.9999993
0.084
0.9067085561598839 0.6227126726749744 0.08432633156493038 test_preds/sgd/swa_1000_v2.npz
(40262400, 11) 0.9999993
0.084
0.9065544716695675 0.6232305222448281 0.08447101325709701 test_preds/sgd/swa_1000_v3.npz
